In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

In [2]:
occ = pd.read_csv('./data/occupancy.csv')

In [5]:
occ.head()

,time,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,2673,2674,2675,2676,2677,2678,2679,2680,2681,2682
0,2022-09-01 00:00:00,4.0,9.0,2.0,10.0,7.0,4.0,4.0,20.0,36.0,...,16.0,1.0,4.0,5.0,6.0,0.0,4.0,2.0,2.0,0.0
1,2022-09-01 01:00:00,7.0,6.0,2.0,6.0,8.0,7.0,5.0,19.0,47.0,...,25.0,2.0,10.0,6.0,7.0,0.0,3.0,3.0,3.0,4.0
2,2022-09-01 02:00:00,7.0,3.0,1.0,10.0,8.0,8.0,5.0,20.0,60.0,...,17.0,1.0,0.0,5.0,8.0,0.0,7.0,3.0,1.0,8.0
3,2022-09-01 03:00:00,8.0,3.0,1.0,10.0,9.0,8.0,5.0,20.0,58.0,...,17.0,1.0,10.0,5.0,8.0,1.0,7.0,3.0,1.0,3.0
4,2022-09-01 04:00:00,6.0,6.0,2.0,8.0,9.0,7.0,5.0,19.0,54.0,...,25.0,2.0,10.0,5.0,7.0,0.0,3.0,3.0,2.0,0.0


In [7]:
# set time as index
occ = (occ
              .set_index(pd.DatetimeIndex(occ['time']))
              .drop('time', axis=1)
              .sort_index())

occ.head()

,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,2673,2674,2675,2676,2677,2678,2679,2680,2681,2682
time,,,,,,,,,,,,,,,,,,,,,
2022-09-01 00:00:00,4.0,9.0,2.0,10.0,7.0,4.0,4.0,20.0,36.0,22.0,...,16.0,1.0,4.0,5.0,6.0,0.0,4.0,2.0,2.0,0.0
2022-09-01 01:00:00,7.0,6.0,2.0,6.0,8.0,7.0,5.0,19.0,47.0,33.0,...,25.0,2.0,10.0,6.0,7.0,0.0,3.0,3.0,3.0,4.0
2022-09-01 02:00:00,7.0,3.0,1.0,10.0,8.0,8.0,5.0,20.0,60.0,29.0,...,17.0,1.0,0.0,5.0,8.0,0.0,7.0,3.0,1.0,8.0
2022-09-01 03:00:00,8.0,3.0,1.0,10.0,9.0,8.0,5.0,20.0,58.0,28.0,...,17.0,1.0,10.0,5.0,8.0,1.0,7.0,3.0,1.0,3.0
2022-09-01 04:00:00,6.0,6.0,2.0,8.0,9.0,7.0,5.0,19.0,54.0,31.0,...,25.0,2.0,10.0,5.0,7.0,0.0,3.0,3.0,2.0,0.0


In [9]:
occ1009 = occ[['1009']]

In [11]:
occ1009

,1009
time,
2022-09-01 00:00:00,36.0
2022-09-01 01:00:00,47.0
2022-09-01 02:00:00,60.0
2022-09-01 03:00:00,58.0
2022-09-01 04:00:00,54.0
...,...
2023-08-31 19:00:00,16.0
2023-08-31 20:00:00,13.0
2023-08-31 21:00:00,13.0


In [15]:
occ1009.isnull().sum()

1009    0
dtype: int64

## Train test split

In [18]:
# split the data temporally, no shuffling, 90% train data, 10% test, 
y_train, y_test = train_test_split(occ1009, test_size=0.1, shuffle=False, random_state=123)

In [20]:
y_train.shape

(7884, 1)

In [22]:
y_test.shape

(876, 1)

In [24]:
y_train.isna().sum()

1009    0
dtype: int64

In [26]:
y_test.isna().sum()

1009    0
dtype: int64

In [28]:
#monthly
y_train = y_train.asfreq('MS') #trying monthly
model_monthly = ARIMA(y_train, order=(1,2,1)).fit()

In [32]:
model_monthly.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                   1009   No. Observations:                   11
Model:                 ARIMA(1, 2, 1)   Log Likelihood                 -40.212
Date:                Mon, 16 Dec 2024   AIC                             86.423
Time:                        23:40:41   BIC                             87.015
Sample:                    09-01-2022   HQIC                            85.146
                         - 07-01-2023                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.3050      0.730     -0.418      0.676      -1.736       1.126
ma.L1         -0.9997    962.752     -0.001      0.999   -1887.958    1885.959
sigma2       323.1652   3.11e+05      0.001      0.999   -6.09e+05     6.1e+05
===================================================================================
Ljung-Box (L1) (Q):                   0.90   Jarque-Bera (JB):                 0.20
Prob(Q):                              0.34   Prob(JB):                         0.90
Heteroskedasticity (H):               0.64   Skew:                             0.34
Prob(H) (two-sided):                  0.72   Kurtosis:                         2.74
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [30]:
model_monthly.aic

86.42316526067572

## Search for ARIMA parameters

In [34]:
from itertools import product
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [39]:
%%time

# Ensure time series index is in datetime format
y_train.index = pd.to_datetime(y_train.index)

# Explicitly set the frequency of the time series data
y_train = y_train.asfreq('MS')  
# 'MS' stands for Month Start; adjust based on your data's frequency

p_array = np.arange(13)

d = 2

q_array = np.arange(13)

results = []
# Initialize an empty list to store results (combinations of p, q, and AIC values)

for p, q in product(p_array, q_array):
    # Iterate over all combinations of 'p' and 'q' values using Cartesian product
    model = ARIMA(y_train, order=(p, d, q)).fit()
    # Instantiate and fit an ARIMA model with the given order (p, d, q) on the training data
    results.append((p, q, model.aic))
    # Append the combination of 'p', 'q', and the AIC value of the model to the results list

LinAlgError: LU decomposition error.